In [55]:
import numpy as np
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from collections import defaultdict
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from autofeat import AutoFeatClassifier
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)
import optuna
from optuna.samplers import TPESampler
from optuna.integration.mlflow import MLflowCallback

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
)

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from dotenv import load_dotenv, find_dotenv
import psycopg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


In [ ]:
# подгружаем .env
load_dotenv()

In [ ]:
TABLE_NAME = "alt_users_churn" # таблица с данными в postgres 
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

# Параметры для трекинга эксперимента
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "churn_experiment_imartnv"
RUN_NAME = "model_bayesian_search" # ваш код здесь
REGISTRY_MODEL_NAME = "churn_model_martynov_alexey"

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

# Настрофка отображения
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

sns.set_style("white")
sns.set_theme(style="whitegrid") 

In [ ]:
# Подключение к базе и получение данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

In [ ]:
#Функция удаления дубликатов
def remove_duplicates(data):
    feature_cols = data.columns.drop('customer_id').tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

In [ ]:
#Функция для заполнения пропусков
def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
    for col in cols_with_nans:
        if data[col].dtype in [float, int]:
            fill_value = data[col].mean()
        elif data[col].dtype == 'object':
            fill_value = data[col].mode().iloc[0]
        data[col] = data[col].fillna(fill_value)
    return data

In [ ]:
#Функция удаления выбросов
def remove_outliers(df: pd.DataFrame, threshold: float = 1.5) -> pd.DataFrame:
        num_cols = df.select_dtypes(include=['float']).columns
        potential_outliers = pd.DataFrame(False, index=df.index, columns=num_cols)
        
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            margin = threshold * IQR
            lower = Q1 - margin
            upper = Q3 + margin
            potential_outliers[col] = ~df[col].between(lower, upper)
        
        outliers = potential_outliers.any(axis=1)
        df_cleaned = df[~outliers]
        return df_cleaned

In [ ]:
#Почистим датасет
df = fill_missing_values(df)
df = remove_duplicates(df)
df = remove_outliers(df)

In [ ]:
df = df.set_index('id')
df = df.drop(columns=['customer_id','begin_date','end_date'])

In [ ]:
obj_df = df.select_dtypes(include="object")
cat_columns = ["type", "payment_method", "internet_service", "gender"]

In [ ]:
num_df = df.select_dtypes(include='float64').dropna()
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

In [ ]:
df.dropna(subset=['monthly_charges','total_charges'],inplace=True)
num_df = df[num_columns]

In [ ]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None


# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_spl.get_feature_names_out(num_columns))
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))

encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

In [ ]:
num_df

In [ ]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('spline', SplineTransformer(n_knots=n_knots, degree=degree_spline), num_columns),
        ('quantile', QuantileTransformer(n_quantiles=n_quantiles), num_columns),
        ('robust', RobustScaler(), num_columns),
        ('polynomial', PolynomialFeatures(degree=degree), num_columns),
        ('kbins', KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample), num_columns),
    ]
)

categorical_transformer = Pipeline(
	steps=[
        ('onehot',OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
	('num',numeric_transformer,num_columns),
    ('cat',categorical_transformer,cat_columns)
    ]
)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df.reset_index(drop=True), transformed_df.reset_index(drop=True)], axis=1)
df.head(2)

In [ ]:
df.head(2)

In [ ]:
# Обучение модели CatBoostClassifier
X = transformed_df
y = df['target']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

In [39]:
def objective(trial: optuna.Trial) -> float:
    # сетка гиперпараметров
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2)
    metrics = defaultdict(list)

    for train_index, val_index in skf.split(X_train, y_train):
    # формируем обучающую и валидационную выборки
        train_x = X_train.iloc[train_index]
        val_x   = X_train.iloc[val_index]
        train_y = y_train.iloc[train_index]
        val_y   = y_train.iloc[val_index]

        model.fit(train_x, train_y)
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]

        tn, err1, fp, err2 = confusion_matrix(val_y, prediction, normalize="all").ravel()
        auc   = roc_auc_score(val_y, probas)
        prec  = precision_score(val_y, prediction)
        rec   = recall_score(val_y, prediction)
        f1    = f1_score(val_y, prediction)
        lloss = log_loss(val_y, prediction)

        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["auc"].append(auc)
        metrics["precision"].append(prec)
        metrics["recall"].append(rec)
        metrics["f1"].append(f1)
        metrics["logloss"].append(lloss)

    # усредняем по фолдам
    err_1     = np.median(metrics["err1"])
    err_2     = np.median(metrics["err2"])
    auc       = np.median(metrics["auc"])
    precision = np.mean(metrics["precision"])
    recall    = np.mean(metrics["recall"])
    f1        = np.mean(metrics["f1"])
    logloss   = np.mean(metrics["logloss"])

    # логируем тэг trial_number в каждом trial-run
    mlflow.set_tag("trial_number", trial.number)

    # можно залогировать агрегированные метрики вручную, но MLflowCallback это сделает за нас
    return auc

In [45]:
# ─── 3) Создаём или получаем эксперимент ────────────────────────────────────
exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if exp is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = int(exp.experiment_id)

In [46]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [47]:
# убеждаемся, что теперь MLflow знает, куда логить
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='s3://s3-student-mle-20241125-59b9e9f709/4', creation_time=1736518697322, experiment_id='4', last_update_time=1736518697322, lifecycle_stage='active', name='churn_experiment_imartnv', tags={}>

In [56]:
# ─── 4) Настраиваем Optuna + MLflowCallback ───────────────────────────────
mlflc = MLflowCallback(
    tracking_uri  = mlflow.get_tracking_uri(),
    metric_name   = "auc",
    create_experiment = False,
    mlflow_kwargs = {
        "experiment_id": experiment_id,
        "nested": True,          # все trial‐run’ы будут вложенными
        "run_name": RUN_NAME, 
        "tags": {MLFLOW_PARENT_RUN_ID: run_id}   # имя родительского запуска
    },
)

/tmp/ipykernel_9163/3918233610.py:2: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(


In [57]:
study = optuna.create_study(
    study_name     = STUDY_NAME,
    storage        = STUDY_DB_NAME,
    sampler        = TPESampler(),
    direction      = "maximize",
    load_if_exists = True,
)

[I 2025-05-11 18:56:46,371] Using an existing study with name 'churn_model' instead of creating a new one.


In [58]:
# ─── 5) Оптимизируем ───────────────────────────────────────────────────────
study.optimize(objective, n_trials=5, callbacks=[mlflc])

[I 2025-05-11 18:57:00,185] Trial 16 finished with value: 0.833507482288539 and parameters: {'learning_rate': 0.08364765716565903, 'depth': 4, 'l2_leaf_reg': 2.275533315286375, 'random_strength': 1.2934637957917392}. Best is trial 13 with value: 0.8391505533730765.


🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/4b5e67e94f044f15b31aed2a85772b16
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 18:57:03,375] Trial 17 finished with value: 0.8364673501093789 and parameters: {'learning_rate': 0.017692718058815816, 'depth': 3, 'l2_leaf_reg': 3.0575871893376725, 'random_strength': 2.1349438541839003}. Best is trial 13 with value: 0.8391505533730765.


🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/d87980072b2c4dfcb3837d6ec7f1334d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 18:57:06,995] Trial 18 finished with value: 0.8369493501585836 and parameters: {'learning_rate': 0.0389401507679321, 'depth': 5, 'l2_leaf_reg': 1.7010015940222294, 'random_strength': 0.6967935450020514}. Best is trial 13 with value: 0.8391505533730765.


🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/83e1cd85e5744efdb85dd9de1c46d4b1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 18:57:08,303] Trial 19 finished with value: 0.8283606644701323 and parameters: {'learning_rate': 0.016378557422773804, 'depth': 1, 'l2_leaf_reg': 3.203459569174675, 'random_strength': 0.11351021719414622}. Best is trial 13 with value: 0.8391505533730765.


🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/876873b8935244bda13b37aecd6b652e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 18:57:10,112] Trial 20 finished with value: 0.8390175385192676 and parameters: {'learning_rate': 0.06143115201937481, 'depth': 3, 'l2_leaf_reg': 2.213467569632706, 'random_strength': 2.382103816403742}. Best is trial 13 with value: 0.8391505533730765.


🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/4bec084131104f409ec9061920f5854d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


In [59]:
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)

    # логируем итоговые метрики
    mlflow.log_metrics({
        "auc":       float(roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])),
        "precision": float(precision_score(y_test, best_model.predict(X_test))),
        "recall":    float(recall_score(y_test, best_model.predict(X_test))),
        "f1":        float(f1_score(y_test, best_model.predict(X_test))),
        "logloss":   float(log_loss(y_test, best_model.predict(X_test))),
    })

    # а потом сохраняем модель
    mlflow.catboost.log_model(best_model, artifact_path="best_model")
    print("Run ID:", run_id)
    print("✅ Готово!")

0:	learn: 0.6680526	total: 3.88ms	remaining: 3.88s
1:	learn: 0.6385991	total: 9.87ms	remaining: 4.92s
2:	learn: 0.6197316	total: 13.8ms	remaining: 4.59s
3:	learn: 0.6031490	total: 18.5ms	remaining: 4.61s
4:	learn: 0.5863837	total: 23.5ms	remaining: 4.67s
5:	learn: 0.5694388	total: 28.4ms	remaining: 4.71s
6:	learn: 0.5573823	total: 33.1ms	remaining: 4.69s
7:	learn: 0.5476143	total: 37.9ms	remaining: 4.7s
8:	learn: 0.5389242	total: 42.5ms	remaining: 4.68s
9:	learn: 0.5315107	total: 47.4ms	remaining: 4.69s
10:	learn: 0.5192152	total: 52.1ms	remaining: 4.68s
11:	learn: 0.5117593	total: 56.8ms	remaining: 4.67s
12:	learn: 0.5057331	total: 61.5ms	remaining: 4.67s
13:	learn: 0.4994828	total: 66ms	remaining: 4.65s
14:	learn: 0.4944561	total: 68.9ms	remaining: 4.53s
15:	learn: 0.4894391	total: 71.1ms	remaining: 4.37s
16:	learn: 0.4823384	total: 73.1ms	remaining: 4.23s
17:	learn: 0.4794181	total: 76ms	remaining: 4.15s
18:	learn: 0.4763694	total: 79.1ms	remaining: 4.08s
19:	learn: 0.4731897	total:

2025/05/11 18:57:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: 5ac13ae97d6f4670811eeeb2dbe099a7
✅ Готово!
🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/e60da20592714e1087573b2bb22a97cc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


In [62]:
with mlflow.start_run(
        run_name=RUN_NAME,
        experiment_id=experiment_id,
        tags={"search_method": "optuna_tpe"}   # общий тэг в parent-run
    ) as parent_run:

    parent_id = parent_run.info.run_id

    mlflc = MLflowCallback(
        tracking_uri     = mlflow.get_tracking_uri(),
        metric_name      = "auc",
        create_experiment = False,
        mlflow_kwargs    = {
            "experiment_id": experiment_id,
            "nested": True,                     # <<< здесь
            "tags": { MLFLOW_PARENT_RUN_ID: parent_id }
        }
    )

    study = optuna.create_study(
        study_name     = STUDY_NAME,
        storage        = STUDY_DB_NAME,
        sampler        = TPESampler(),
        direction      = "maximize",
        load_if_exists = True,
    )

    study.optimize(objective, n_trials=10, callbacks=[mlflc])

    best_params = study.best_params
    print(f"Finished {len(study.trials)} trials. Best params:\n{best_params}")

    # 5) Логируем лучшую модель и её тэги в том же parent-run
    best_model = CatBoostClassifier(**best_params)
    best_model.fit(X_train, y_train)

    mlflow.set_tag("final_model", "true")  # тэг для финальной модели  
    mlflow.catboost.log_model(
        cb_model=model,, 
        artifact_path="cv",
        registered_model_name=REGISTRY_MODEL_NAME
    )

    print("Run ID:", parent_id)
    print("✅ Тэги теперь будут в таблице tags!")

/tmp/ipykernel_9163/3872117586.py:9: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-05-11 19:07:23,949] Using an existing study with name 'churn_model' instead of creating a new one.
[I 2025-05-11 19:07:25,986] Trial 32 finished with value: 0.8382050601868054 and parameters: {'learning_rate': 0.02015732073683575, 'depth': 2, 'l2_leaf_reg': 0.13280608346028488, 'random_strength': 0.9464661751225636}. Best is trial 31 with value: 0.8392743633817229.


🏃 View run 32 at: http://127.0.0.1:5000/#/experiments/4/runs/d597d283324b4ca3844589254b920506
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 19:07:27,224] Trial 33 finished with value: 0.8342417613531556 and parameters: {'learning_rate': 0.03058134030988036, 'depth': 1, 'l2_leaf_reg': 1.3329533449264386, 'random_strength': 0.5035818441272865}. Best is trial 31 with value: 0.8392743633817229.


🏃 View run 33 at: http://127.0.0.1:5000/#/experiments/4/runs/7d474017e4804a79a6d3c894e542342d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 19:07:29,493] Trial 34 finished with value: 0.839699985432704 and parameters: {'learning_rate': 0.033599549802624584, 'depth': 4, 'l2_leaf_reg': 3.588966490568893, 'random_strength': 0.9305169127194608}. Best is trial 34 with value: 0.839699985432704.


...

🏃 View run 40 at: http://127.0.0.1:5000/#/experiments/4/runs/7a6dc6322932404e81ce841cc4c2940d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-05-11 19:08:32,600] Trial 41 finished with value: 0.8355504081702865 and parameters: {'learning_rate': 0.03535833399646549, 'depth': 7, 'l2_leaf_reg': 4.8270023585564985, 'random_strength': 0.31547432925234}. Best is trial 35 with value: 0.8406909935947927.


🏃 View run 41 at: http://127.0.0.1:5000/#/experiments/4/runs/f2dd71ac52f34f43b1147183590543f3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
Finished 42 trials. Best params:
{'learning_rate': 0.03323439384086525, 'depth': 4, 'l2_leaf_reg': 3.728494926073251, 'random_strength': 0.9602975790236736}
0:	learn: 0.6747875	total: 3.9ms	remaining: 3.9s
1:	learn: 0.6584147	total: 7.52ms	remaining: 3.75s
2:	learn: 0.6396824	total: 11.1ms	remaining: 3.7s
3:	learn: 0.6258475	total: 14.8ms	remaining: 3.68s
4:	learn: 0.6110854	total: 18.4ms	remaining: 3.66s
5:	learn: 0.5954527	total: 21.8ms	remaining: 3.62s
6:	learn: 0.5820048	total: 25.7ms	remaining: 3.64s
7:	learn: 0.5689238	total: 29ms	remaining: 3.6s
8:	learn: 0.5601149	total: 32.8ms	remaining: 3.61s
9:	learn: 0.5516869	total: 36.3ms	remaining: 3.6s
10:	learn: 0.5431889	total: 40ms	remaining: 3.6s
11:	learn: 0.5339477	total: 43.6ms	remaining: 3.59s
12:	learn: 0.5271272	total: 47.1ms	remaining: 3.58s
13:	learn: 0.5208911	total: 50.6m

2025/05/11 19:08:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: ae5284bd4e8f4f89b0162924e7796178
✅ Тэги теперь будут в таблице tags!
🏃 View run model_bayesian_search at: http://127.0.0.1:5000/#/experiments/4/runs/ae5284bd4e8f4f89b0162924e7796178
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
